# Загрузка датасета и предобработка

In [1]:
# импорт модулей
import sqlite3
import pandas as pd

In [2]:
# Считывание данных
data = pd.read_csv('https://raw.githubusercontent.com/ilnurav/real_estate_market_analysis/refs/heads/main/kc_house_data.csv')

data

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [3]:
# Проверка на пропуски и дубликаты
missing_values = data.isnull().sum()
duplicates = data.duplicated().sum()

# Проверка на некорректные значения
incorrect_values = data[data['price'] < 0]

# Вывод результатов проверки
print("Пропуски в данных:\n", missing_values)
print("Число дубликатов:", duplicates)
print("Число некорректных значений:\n", len(incorrect_values))
print("Размерность датасета:", data.shape)

Пропуски в данных:
 id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64
Число дубликатов: 0
Число некорректных значений:
 0
Размерность датасета: (21613, 21)


In [4]:
# Выводим типы данных столбцов
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [5]:
# вывод первых 5 строк поля date
print(data['date'].head())

0    20141013T000000
1    20141209T000000
2    20150225T000000
3    20141209T000000
4    20150218T000000
Name: date, dtype: object


In [6]:
# Преобразование данных: поле date в формат даты
data['date'] = pd.to_datetime(data['date'], format='%Y%m%dT%H%M%S')

# Проверка результата
print(data['date'].head())

0   2014-10-13
1   2014-12-09
2   2015-02-25
3   2014-12-09
4   2015-02-18
Name: date, dtype: datetime64[ns]


# Создание базы данных SQLITE из датафрейма

In [7]:
# Создание подключения к SQLite базе данных
conn = sqlite3.connect('kc_house_database.db')

# Запись данных в SQLite
data.to_sql('kc_house_data', conn, if_exists='replace', index=False)

# Закрытие соединения
conn.close()

# Запрос для получения среднего количества ванных комнат в зависимости от состояния дома


In [8]:
# Подключение к базе данных
conn = sqlite3.connect('kc_house_database.db')
cursor = conn.cursor()

# SQL-запрос
query = """
SELECT
    condition, -- состояние дома
    AVG(bathrooms) AS avg_bathrooms -- среднее количество ванных комнат
FROM
    kc_house_data -- указываем таблицу данных о домах
GROUP BY
    condition -- группируем результаты по состоянию
ORDER BY
    condition; -- сортируем по состоянию для удобства чтения
"""

# Выполнение запроса и получение результата
cursor.execute(query)
rows = cursor.fetchall()

# Закрытие соединения
cursor.close()
conn.close()

# Вывод результата
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(pd.DataFrame(rows, columns=('Состояние дома', 'Среднее количество ванных комнат')))

   Состояние дома  Среднее количество ванных комнат
0               1                          1.175000
1               2                          1.449128
2               3                          2.222632
3               4                          1.900423
4               5                          2.024397


# Количество домов, проданных в 2014 году

In [9]:
import sqlite3
import pandas as pd

# Подключение к базе данных
conn = sqlite3.connect('kc_house_database.db')
cursor = conn.cursor()

# SQL-запрос
query = """
SELECT
    COUNT(*) AS total_houses_sold -- Подсчитываем общее количество проданных домов
FROM
    kc_house_data -- указываем таблицу, из которой выбираем данные
WHERE
    strftime('%Y', date) = '2014' -- указываем условие для года
"""

# Выполнение запроса и получение результата
cursor.execute(query)
rows = cursor.fetchall()

# Закрытие соединения
cursor.close()
conn.close()

# Вывод результата
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(pd.DataFrame(rows, columns=('Общее количество проданных домов',)))

   Общее количество проданных домов
0                             14633


# Топ-10 почтовых индексов с самой высокой средней ценой продажи


In [10]:
# Подключение к базе данных
conn = sqlite3.connect('kc_house_database.db')
cursor = conn.cursor()

# SQL-запрос
query = """
SELECT
    zipcode, -- Почтовый индекс
    AVG(price) AS average_price -- Средняя цена продажи данного почтового индекса
FROM
    kc_house_data -- Исходная таблица с данными о домах
GROUP BY
    zipcode -- Группируем результаты по почтовому индексу
ORDER BY
    average_price DESC -- Сортируем результаты по средней цене в порядке убывания
LIMIT 10; -- Ограничиваем результат первыми 10 записями
"""

# Выполнение запроса и получение результата
cursor.execute(query)
rows = cursor.fetchall()

# Закрытие соединения
cursor.close()
conn.close()

# Вывод результата
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(pd.DataFrame(rows, columns=('Почтовый индекс', 'Средняя цена продажи')))

   Почтовый индекс  Средняя цена продажи
0            98039          2.160607e+06
1            98004          1.355927e+06
2            98040          1.194230e+06
3            98112          1.095499e+06
4            98102          9.012583e+05
5            98109          8.796236e+05
6            98105          8.628252e+05
7            98006          8.596848e+05
8            98119          8.494480e+05
9            98005          8.101649e+05


# Список домов с площадью более 300 кв.м. и ценой продаж менее 500,000 долларов

In [11]:
# Подключение к базе данных
conn = sqlite3.connect('kc_house_database.db')
cursor = conn.cursor()

# SQL-запрос с LIMIT
query = """
SELECT
    id,
    date,
    price,
    bedrooms,
    bathrooms,
    sqft_living,
    sqft_lot,
    floors,
    waterfront,
    view,
    condition,
    grade,
    sqft_above,
    sqft_basement,
    yr_built,
    yr_renovated,
    zipcode,
    lat,
    long,
    sqft_living15,
    sqft_lot15
FROM
    kc_house_data
WHERE
    sqft_living > 300
    AND price < 500000
ORDER BY
    price ASC
LIMIT 10; -- Ограничение на 10 записей
"""

# Выполнение запроса и получение результата
cursor.execute(query)
rows = cursor.fetchall()

# Закрытие соединения
cursor.close()
conn.close()

# Вывод результата
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(pd.DataFrame(rows, columns=[
        'ID', 'Дата продажи', 'Цена', 'Количество спален', 'Количество ванных комнат',
        'Площадь жилого пространства', 'Площадь земельного участка', 'Количество этажей',
        'Водный фронт', 'Оценка вида', 'Состояние', 'Оценка качества',
        'Площадь над землёй', 'Площадь подвала', 'Год постройки',
        'Год последнего ремонта', 'Почтовый индекс', 'Широта', 'Долгота',
        'Средняя площадь 15 ближайших домов', 'Средняя площадь земельного участка 15 ближайших домов'
    ]))

           ID         Дата продажи     Цена  Количество спален  \
0  3421079032  2015-02-17 00:00:00  75000.0                  1   
1    40000362  2014-05-06 00:00:00  78000.0                  2   
2  8658300340  2014-05-23 00:00:00  80000.0                  1   
3  3028200080  2015-03-24 00:00:00  81000.0                  2   
4  3883800011  2014-11-05 00:00:00  82000.0                  3   
5  1623049041  2014-05-08 00:00:00  82500.0                  2   
6  7999600180  2014-05-29 00:00:00  83000.0                  2   
7  1523049188  2015-04-30 00:00:00  84000.0                  2   
8  2422049104  2014-09-15 00:00:00  85000.0                  2   
9  1322049150  2015-03-05 00:00:00  85000.0                  2   

   Количество ванных комнат  Площадь жилого пространства  \
0                      0.00                          670   
1                      1.00                          780   
2                      0.75                          430   
3                      1.00      

# TOP 10 домов с самым высоким соотношением цены к площади жилого пространства

In [12]:
# Подключение к базе данных
conn = sqlite3.connect('kc_house_database.db')
cursor = conn.cursor()

# SQL-запрос
query = """
SELECT
    id,               -- уникальный идентификатор дома
    price,            -- цена дома
    sqft_living,     -- площадь жилого пространства
    price / sqft_living AS price_per_sqft -- вычисляем соотношение цена/площадь
FROM
    kc_house_data -- выбираем данные из таблицы kc_house_data
ORDER BY
    price_per_sqft DESC -- сортируем по соотношению цена/площадь в порядке убывания
LIMIT 10; -- ограничиваем выборку первыми 10 записями
"""

# Выполнение запроса и получение результата
cursor.execute(query)
rows = cursor.fetchall()

# Закрытие соединения
cursor.close()
conn.close()

# Вывод результата
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(pd.DataFrame(rows, columns=[
        'ID', 'Цена', 'Площадь жилого пространства', 'соотношение цена/площадь'
    ]))

           ID       Цена  Площадь жилого пространства  \
0  6021500970   874950.0                         1080   
1   724069059  2400000.0                         3000   
2  1118000320  3400000.0                         4260   
3  6303400395   325000.0                          410   
4  4389200610   903000.0                         1140   
5  3222049087   570000.0                          720   
6  4131900066  3100000.0                         3920   
7  5536100010  1050000.0                         1330   
8  2767603026   425000.0                          540   
9  9808700025  1500000.0                         1910   

   соотношение цена/площадь  
0                810.138889  
1                800.000000  
2                798.122066  
3                792.682927  
4                792.105263  
5                791.666667  
6                790.816327  
7                789.473684  
8                787.037037  
9                785.340314  


# Список домов с ценой рыночной стоимости

In [13]:
import sqlite3
import pandas as pd

# Подключение к базе данных
conn = sqlite3.connect('kc_house_database.db')
cursor = conn.cursor()

# SQL-запрос
query = """
SELECT
    id,                     -- уникальный идентификатор дома
    date,                   -- дата продажи дома
    price,                  -- цена продажи дома
    bedrooms,               -- количество спален
    bathrooms,              -- количество ванных комнат
    sqft_living,           -- площадь жилого пространства
    sqft_lot,              -- площадь земельного участка
    floors,                 -- количество этажей
    waterfront,             -- признак наличия водного фронта (0 - нет, 1 - да)
    view,                   -- оценка вида из дома (0 - нет вида, 4 - лучший вид)
    condition,              -- общее состояние дома (от 1 до 5)
    grade,                  -- оценка качества материалов и отделки (от 1 до 13)
    sqft_above,            -- площадь жилого пространства над землёй
    sqft_basement,         -- площадь подвала
    yr_built,              -- год постройки дома
    yr_renovated,          -- год последнего ремонта (0 - не было ремонта)
    zipcode,               -- почтовый индекс
    lat,                   -- широта местоположения дома
    long,                  -- долгота местоположения дома
    sqft_living15,         -- средняя площадь жилого пространства 15 ближайших домов
    sqft_lot15             -- средняя площадь земельного участка 15 ближайших домов
FROM kc_house_data
WHERE price < (
    SELECT AVG(price)
    FROM kc_house_data
)
ORDER BY price ASC
LIMIT 10; -- Ограничение на 10 результатов
"""

# Выполнение запроса и получение результата
cursor.execute(query)
rows = cursor.fetchall()

# Закрытие соединения
cursor.close()
conn.close()

# Вывод результата
columns = [
    'ID', 'Дата продажи', 'Цена', 'Спальни', 'Ванные комнаты',
    'Площадь жилого пространства', 'Площадь земельного участка',
    'Этажи', 'Водный фронт', 'Вид',
    'Состояние', 'Качество', 'Площадь над землёй',
    'Площадь подвала', 'Год постройки',
    'Год ремонта', 'Почтовый индекс',
    'Широта', 'Долгота',
    'Средняя площадь жилого пространства 15 ближайших домов',
    'Средняя площадь земельного участка 15 ближайших домов'
]

# Вывод результата в виде DataFrame
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(pd.DataFrame(rows, columns=columns))

           ID         Дата продажи     Цена  Спальни  Ванные комнаты  \
0  3421079032  2015-02-17 00:00:00  75000.0        1            0.00   
1    40000362  2014-05-06 00:00:00  78000.0        2            1.00   
2  8658300340  2014-05-23 00:00:00  80000.0        1            0.75   
3  3028200080  2015-03-24 00:00:00  81000.0        2            1.00   
4  3883800011  2014-11-05 00:00:00  82000.0        3            1.00   
5  1623049041  2014-05-08 00:00:00  82500.0        2            1.00   
6  7999600180  2014-05-29 00:00:00  83000.0        2            1.00   
7  1523049188  2015-04-30 00:00:00  84000.0        2            1.00   
8  2422049104  2014-09-15 00:00:00  85000.0        2            1.00   
9  1322049150  2015-03-05 00:00:00  85000.0        2            1.00   

   Площадь жилого пространства  Площадь земельного участка  Этажи  \
0                          670                       43377    1.0   
1                          780                       16344    1.0   


# Список почтовых индексов с наибольшим количеством домов на продажу, отсортированный по убыванию количества

In [14]:
import sqlite3
import pandas as pd

# Подключение к базе данных
conn = sqlite3.connect('kc_house_database.db')
cursor = conn.cursor()

# SQL-запрос
query = """
SELECT
    zipcode AS house_count, -- почтовый индекс
    COUNT(id) AS count      -- количество домов для каждого почтового индекса
FROM
    kc_house_data          -- указываем таблицу данных о домах
GROUP BY
    zipcode                -- группируем результаты по почтовым индексам
ORDER BY
    count DESC;            -- сортируем по убыванию количества домов
"""

# Выполнение запроса и получение результата
cursor.execute(query)
rows = cursor.fetchall()

# Закрытие соединения
cursor.close()
conn.close()

# Вывод результата
columns = ['Почтовый индекс', 'Количество домов']
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(pd.DataFrame(rows, columns=columns))

    Почтовый индекс  Количество домов
0             98103               602
1             98038               590
2             98115               583
3             98052               574
4             98117               553
5             98042               548
6             98034               545
7             98118               508
8             98023               499
9             98006               498
10            98133               494
11            98059               468
12            98058               455
13            98155               446
14            98074               441
15            98033               432
16            98027               412
17            98125               410
18            98056               406
19            98053               405
20            98001               362
21            98075               359
22            98126               354
23            98092               351
24            98144               343
25          

# Вывод списка почтовых индексов с максимальной и минимальной ценой продаж домов


In [15]:
# Подключение к базе данных
conn = sqlite3.connect('kc_house_database.db')
cursor = conn.cursor()

# SQL-запрос
query = """
SELECT
    zipcode, -- почтовый индекс
    MAX(price) AS max_price, -- максимальная цена
    MIN(price) AS min_price -- минимальная цена
FROM
    kc_house_data -- указываем таблицу данных о домах
GROUP BY
    zipcode -- группируем результаты по почтовому индексу
ORDER BY
    max_price DESC, min_price ASC; -- сортируем по максимальной и минимальной цене
"""

# Выполнение запроса и получение результата
cursor.execute(query)
rows = cursor.fetchall()

# Закрытие соединения
cursor.close()
conn.close()

# Вывод результата
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(pd.DataFrame(rows, columns=('Почтовый индекс', 'Максимальная цена', 'Минимальная цена')))

    Почтовый индекс  Максимальная цена  Минимальная цена
0             98102          7700000.0          330000.0
1             98004          7062500.0          425000.0
2             98039          6885000.0          787500.0
3             98040          5300000.0          500000.0
4             98033          5110800.0          130000.0
5             98155          4500000.0          153000.0
6             98006          4208000.0          247500.0
7             98177          3800000.0          245560.0
8             98105          3650000.0          380000.0
9             98144          3600000.0          181000.0
10            98008          3567000.0          250000.0
11            98112          3400000.0          169317.0
12            98109          3200000.0          216650.0
13            98199          3200000.0          323000.0
14            98075          3200000.0          406430.0
15            98034          3120000.0           90000.0
16            98074          30

# Запрос для подсчета количества домов с ремонтом и без ремонта


In [16]:
# Подключение к базе данных
conn = sqlite3.connect('kc_house_database.db')
cursor = conn.cursor()

# SQL-запрос
query = """
SELECT
    COUNT(CASE WHEN yr_renovated > 0 THEN 1 END) AS homes_with_renovation, -- подсчитываем количество домов с ремонтом
    COUNT(CASE WHEN yr_renovated = 0 THEN 1 END) AS homes_without_renovation, -- подсчитываем количество домов без ремонта
    CAST(COUNT(CASE WHEN yr_renovated > 0 THEN 1 END) AS float) /
    NULLIF(COUNT(CASE WHEN yr_renovated = 0 THEN 1 END), 0) AS renovation_ratio -- отношение домов с ремонтом к домам без ремонта
FROM
    kc_house_data; -- указываем таблицу, из которой будем брать данные
"""

# Выполнение запроса и получение результата
cursor.execute(query)
rows = cursor.fetchall()

# Закрытие соединения
cursor.close()
conn.close()

# Вывод результата
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(pd.DataFrame(rows, columns=('Количество домов с ремонтом', 'Количество домов без ремонта', 'Отношение ремонтов')))

   Количество домов с ремонтом  Количество домов без ремонта  \
0                          914                         20699   

   Отношение ремонтов  
0            0.044157  


# Запрос для получения средней площади жилого пространства по состоянию дома


In [17]:
# Подключение к базе данных
conn = sqlite3.connect('kc_house_database.db')
cursor = conn.cursor()

# SQL-запрос
query = """
SELECT
    condition, -- состояние дома
    AVG(sqft_living) AS avg_living_area -- средняя площадь жилого пространства
FROM
    kc_house_data -- указываем таблицу данных о домах
GROUP BY
    condition -- группируем результаты по состоянию
ORDER BY
    condition; -- сортируем по состоянию для удобства чтения
"""

# Выполнение запроса и получение результата
cursor.execute(query)
rows = cursor.fetchall()

# Закрытие соединения
cursor.close()
conn.close()

# Вывод результата
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(pd.DataFrame(rows, columns=('Состояние дома', 'Средняя площадь жилого пространства')))

   Состояние дома  Средняя площадь жилого пространства
0               1                          1216.000000
1               2                          1410.058140
2               3                          2149.042050
3               4                          1950.991724
4               5                          2022.911229
